In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from corner import corner
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp

tfk = tf.keras
tfd = tfp.distributions
tfb = tfp.bijectors

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
def powerlaw(x, slope, lo, hi):
    
    return (
        (x >= lo) * (x <= hi) 
        * x**slope * (slope+1) 
        / (hi**(slope+1) - lo**(slope+1))
        )

def sample_powerlaw(n_samples, slope, lo, hi):
    
    return (
        np.random.uniform(size=n_samples) * (hi**(slope+1) - lo**(slope+1))
        + lo**(slope+1)
        )**(1/(slope+1))

In [ ]:
n_dim = 2
n_samples = 10000
slope = 1
lo = 0
hi = 1

data = sample_powerlaw([n_samples, n_dim], 1, 0, 1)

corner(data);

In [ ]:
def all_permuted(n_dim):
    
    permutations = np.array(list(itertools.permutations(range(n_dim))))
    permuted = ~np.any(permutations == list(range(n_dim)), axis=1)
    
    return permutations[permuted]

In [ ]:
permutations = all_permuted(n_dim)
permutations

In [ ]:
n_flows = 10
n_layers = 1
n_neurons = 1024

# The function u = f(x), where x is the data and u is the base variate
bijectors = []

# We transform at the end with a logistic function
# This ensures all samples are in [0, 1]
bijectors.append(tfb.Scale(scale=.5))
bijectors.append(tfb.Shift(shift=1.))
bijectors.append(tfb.Tanh())

for i in range(n_flows):
    bijectors.append(tfb.MaskedAutoregressiveFlow(tfb.AutoregressiveNetwork(
        params=2,
        hidden_units=[n_neurons]*n_layers,
        activation='relu',
        ))
        )
    #bijectors.append(tfb.BatchNormalization(training=True))
    bijectors.append(tfb.Permute(list(reversed(range(n_dim)))))

bijector = tfb.Chain(bijectors)
distribution = tfd.MultivariateNormalDiag(loc=[0]*n_dim)
nf = tfd.TransformedDistribution(distribution=distribution, bijector=bijector)

In [ ]:
# Check it's bounded
nf.sample(10000).numpy().min(), nf.sample(10000).numpy().max()

In [ ]:
epochs = 10
batch_size = 100
learning_rate = 1e-3

x = tf.keras.Input(shape=[n_dim], dtype=tf.float32)
log_prob = nf.log_prob(x)

model = tf.keras.Model(
    inputs=x,
    outputs=log_prob,
    )

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
    loss=lambda _, log_prob: -log_prob,
    )

steps_per_epoch = n_samples // batch_size

result = model.fit(
    x=data, y=np.zeros(n_samples),
    epochs=epochs, 
    batch_size=batch_size,
    steps_per_epoch=steps_per_epoch,
    shuffle=True,
    verbose=1,
    )

In [ ]:
plt.plot(result.history['loss']);

In [ ]:
# Should be like the training data
corner(nf.sample(n_samples).numpy(), truths=[1]*n_dim);

In [ ]:
# Can I call bijector directly?
corner(
    nf.bijector.forward(distribution.sample(10000)).numpy(),
    );

In [ ]:
# Should be a Gaussian
fig = corner(
    nf.bijector.inverse(
        sample_powerlaw([n_samples, n_dim], slope, lo, hi).astype(np.float32),
        ).numpy(),
    range=[[-4, 4]]*n_dim,
    );

In [ ]:
if n_dim == 1:
    
    points = np.linspace(0, 1, 200)
    probs = np.exp(nf.log_prob(points[:, None]))
    
    plt.plot(points, powerlaw(points, slope, lo, hi))
    plt.plot(points, probs);

elif n_dim == 2:

    points = np.linspace(-.5, 1.5, 200)
    axes = np.meshgrid(*[points]*n_dim)
    grid = np.concatenate([ax.reshape(-1, 1) for ax in axes], axis=1)

    probs = np.exp(nf.log_prob(grid)).reshape(axes[0].shape)

    plt.imshow(
        probs,
        aspect='equal',
        origin='lower',
        extent=[-.5, 1.5, -.5, 1.5],
        );